In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, \
  ConfusionMatrixDisplay, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, matthews_corrcoef

In [2]:
data = pd.read_csv(r'..\..\10_fold_cross_validation\train_10folds.csv')
data

,MaxPartialCharge,FpDensityMorgan2,BCUT2D_CHGLO,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA3,SlogP_VSA3,SlogP_VSA8,EState_VSA6,NumHAcceptors,NumSaturatedCarbocycles,fr_bicyclic,TARGET,Kfold
0,0.335201,1.714286,-2.072068,5.975550,0.000000,12.132734,9.551078,0.000000,38.974594,43.638476,6,0,1,1.0,0
1,0.226791,2.000000,-2.362642,7.150190,5.948339,0.000000,9.967957,0.000000,10.440599,11.336786,6,0,0,1.0,1
2,0.211302,1.772727,-2.089721,7.912349,9.837253,29.297126,4.983979,9.837253,10.902925,24.265468,4,0,1,0.0,1
3,0.226898,1.869565,-2.357502,6.433493,5.948339,35.334614,15.284746,0.000000,11.126903,12.263211,5,0,0,0.0,0
4,0.158370,1.870968,-2.421374,7.991366,0.000000,23.362825,19.935914,6.420822,0.000000,18.460054,9,0,1,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.152613,2.000000,-2.415835,6.432407,0.000000,35.334614,9.967957,11.343745,11.257379,6.066367,6,0,0,1.0,3
2735,0.335203,1.700000,-2.001320,6.164147,0.000000,30.331835,4.983979,0.000000,10.902925,36.528679,2,0,1,0.0,6
2736,0.264178,1.833333,-2.414068,5.737009,5.948339,38.112943,9.551078,11.784535,11.126903,11.614772,7,0,0,1.0,8
2737,0.338995,1.815789,-2.065301,7.218862,5.907180,59.014740,24.544948,18.386966,43.634305,6.196844,9,0,2,0.0,5


In [3]:
def run(fold, data):
    # load the full training data with folds
    df = data
    # all columns are features except target and kfold columns
    features = [
        f for f in df.columns if f not in ("TARGET", "Kfold")
    ]
    # get training data using folds
    df_train = df[df.Kfold != fold].reset_index(drop=True)
    # get validation data using folds
    df_valid = df[df.Kfold == fold].reset_index(drop=True)
    # get training data
    X_train = df_train[features].values
    # get validation data
    X_valid = df_valid[features].values
    # initialize Logistic Regression model
    model = KNeighborsClassifier()
    model.fit(X_train, df_train.TARGET.values)
    valid_preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(df_valid.TARGET.values, valid_preds)
    y_pred = model.predict(X_valid)
    y_true = df_valid.TARGET.values
    accuracy = accuracy_score(y_true,y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model

In [4]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []

for fold_ in range(10):
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model = run(fold_, data)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    filename = 'KNN_' + str(fold_) + '.pkl'
    joblib.dump(model, filename)
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

Fold = 0, AUC = 0.9285048634166231, Accuracy = 0.8613138686131386,           Precision_1 = 0.8390804597701149, Precision_0 = 0.8716577540106952          Recall_1 = 0.7525773195876289, Recall_0 = 0.9209039548022598, F1Score = 0.7934782608695652, kappa = 0.6895461864154093, MCC = 0.6918590015130924
Fold = 1, AUC = 0.9133321684431241, Accuracy = 0.8576642335766423,           Precision_1 = 0.8085106382978723, Precision_0 = 0.8833333333333333          Recall_1 = 0.7835051546391752, Recall_0 = 0.8983050847457628, F1Score = 0.7958115183246073, kappa = 0.6866091852894598, MCC = 0.6868087826425187
Fold = 2, AUC = 0.9001980313355465, Accuracy = 0.8138686131386861,           Precision_1 = 0.7446808510638298, Precision_0 = 0.85          Recall_1 = 0.7216494845360825, Recall_0 = 0.864406779661017, F1Score = 0.7329842931937173, kappa = 0.5901812423016013, MCC = 0.5903528080428005
Fold = 3, AUC = 0.923466713262275, Accuracy = 0.8613138686131386,           Precision_1 = 0.7920792079207921, Precision_0

In [5]:
fold_metrics = pd.DataFrame(columns=['Accuracy','AUC','Precision_1','Precision_0','Recall_1','Recall_0','F1score','Kappa','MCC'])
fold_metrics['Accuracy'] = np.array(accuracies)
fold_metrics['AUC'] = np.array(aucs)
fold_metrics['Precision_1'] = np.array(precisions_1)
fold_metrics['Precision_0'] = np.array(precisions_0)
fold_metrics['Recall_1'] = np.array(recalls_1)
fold_metrics['Recall_0'] = np.array(recalls_0)
fold_metrics['F1score'] = np.array(f1scores)
fold_metrics['Kappa'] = np.array(kappas)
fold_metrics['MCC'] = np.array(MCCs)
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.861314,0.928505,0.839080,0.871658,0.752577,0.920904,0.793478,0.689546,0.691859
1,0.857664,0.913332,0.808511,0.883333,0.783505,0.898305,0.795812,0.686609,0.686809
2,0.813869,0.900198,0.744681,0.850000,0.721649,0.864407,0.732984,0.590181,0.590353
3,0.861314,0.923467,0.792079,0.901734,0.824742,0.881356,0.808081,0.699579,0.699929
4,0.854015,0.913623,0.793814,0.887006,0.793814,0.887006,0.793814,0.680820,0.680820
5,0.894161,0.936630,0.861702,0.911111,0.835052,0.926554,0.848168,0.766966,0.767189
6,0.864964,0.926117,0.806122,0.897727,0.814433,0.892655,0.810256,0.705445,0.705467
7,0.835766,0.897257,0.754902,0.883721,0.793814,0.858757,0.773869,0.645057,0.645560
8,0.868613,0.927551,0.783019,0.922619,0.864583,0.870787,0.821782,0.718139,0.720351
9,0.864469,0.935205,0.792079,0.906977,0.833333,0.881356,0.812183,0.706272,0.706829


In [6]:
fold_metrics.loc[10,:] = [np.mean(np.array(accuracies)), np.mean(np.array(aucs)), np.mean(np.array(precisions_1)),
                               np.mean(np.array(precisions_0)), np.mean(np.array(recalls_1)), np.mean(np.array(recalls_0)),
                            np.mean(np.array(f1scores)), np.mean(np.array(kappas)), np.mean(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.861314,0.928505,0.839080,0.871658,0.752577,0.920904,0.793478,0.689546,0.691859
1,0.857664,0.913332,0.808511,0.883333,0.783505,0.898305,0.795812,0.686609,0.686809
2,0.813869,0.900198,0.744681,0.850000,0.721649,0.864407,0.732984,0.590181,0.590353
3,0.861314,0.923467,0.792079,0.901734,0.824742,0.881356,0.808081,0.699579,0.699929
4,0.854015,0.913623,0.793814,0.887006,0.793814,0.887006,0.793814,0.680820,0.680820
5,0.894161,0.936630,0.861702,0.911111,0.835052,0.926554,0.848168,0.766966,0.767189
6,0.864964,0.926117,0.806122,0.897727,0.814433,0.892655,0.810256,0.705445,0.705467
7,0.835766,0.897257,0.754902,0.883721,0.793814,0.858757,0.773869,0.645057,0.645560
8,0.868613,0.927551,0.783019,0.922619,0.864583,0.870787,0.821782,0.718139,0.720351
9,0.864469,0.935205,0.792079,0.906977,0.833333,0.881356,0.812183,0.706272,0.706829


In [7]:
fold_metrics.loc[11,:] = [np.std(np.array(accuracies)), np.std(np.array(aucs)), np.std(np.array(precisions_1)),
                               np.std(np.array(precisions_0)), np.std(np.array(recalls_1)), np.std(np.array(recalls_0)),
                            np.std(np.array(f1scores)), np.std(np.array(kappas)), np.std(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.861314,0.928505,0.839080,0.871658,0.752577,0.920904,0.793478,0.689546,0.691859
1,0.857664,0.913332,0.808511,0.883333,0.783505,0.898305,0.795812,0.686609,0.686809
2,0.813869,0.900198,0.744681,0.850000,0.721649,0.864407,0.732984,0.590181,0.590353
3,0.861314,0.923467,0.792079,0.901734,0.824742,0.881356,0.808081,0.699579,0.699929
4,0.854015,0.913623,0.793814,0.887006,0.793814,0.887006,0.793814,0.680820,0.680820
5,0.894161,0.936630,0.861702,0.911111,0.835052,0.926554,0.848168,0.766966,0.767189
6,0.864964,0.926117,0.806122,0.897727,0.814433,0.892655,0.810256,0.705445,0.705467
7,0.835766,0.897257,0.754902,0.883721,0.793814,0.858757,0.773869,0.645057,0.645560
8,0.868613,0.927551,0.783019,0.922619,0.864583,0.870787,0.821782,0.718139,0.720351
9,0.864469,0.935205,0.792079,0.906977,0.833333,0.881356,0.812183,0.706272,0.706829


In [8]:
fold_metrics.index = ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7','Fold_8','Fold_9','Mean','Std']
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
Fold_0,0.861314,0.928505,0.839080,0.871658,0.752577,0.920904,0.793478,0.689546,0.691859
Fold_1,0.857664,0.913332,0.808511,0.883333,0.783505,0.898305,0.795812,0.686609,0.686809
Fold_2,0.813869,0.900198,0.744681,0.850000,0.721649,0.864407,0.732984,0.590181,0.590353
Fold_3,0.861314,0.923467,0.792079,0.901734,0.824742,0.881356,0.808081,0.699579,0.699929
Fold_4,0.854015,0.913623,0.793814,0.887006,0.793814,0.887006,0.793814,0.680820,0.680820
Fold_5,0.894161,0.936630,0.861702,0.911111,0.835052,0.926554,0.848168,0.766966,0.767189
Fold_6,0.864964,0.926117,0.806122,0.897727,0.814433,0.892655,0.810256,0.705445,0.705467
Fold_7,0.835766,0.897257,0.754902,0.883721,0.793814,0.858757,0.773869,0.645057,0.645560
Fold_8,0.868613,0.927551,0.783019,0.922619,0.864583,0.870787,0.821782,0.718139,0.720351
Fold_9,0.864469,0.935205,0.792079,0.906977,0.833333,0.881356,0.812183,0.706272,0.706829


# Final Model

In [9]:
X = data.iloc[:,:-2]
y = data.iloc[:,-2]

In [10]:
model = KNeighborsClassifier()
model.fit(X, y)

KNeighborsClassifier()

In [11]:
joblib.dump(model, 'KNN_Final.pkl')

['KNN_Final.pkl']